In [1]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
from keras.applications import vgg16
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
base_image_path = 'katy.jpg'
style_reference_image_path = 'awesomestyle.jpg'

iterations = 1

In [3]:
img_nrows = 400; img_ncols = 400

In [4]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [5]:
# util function to convert a tensor into a valid image
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x


In [6]:

base_image = K.variable(preprocess_image(base_image_path))

x = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))


In [7]:
random_pixels = np.random.randint(256, size=(img_nrows, img_ncols, 3))
combination_image = preprocess_input(np.expand_dims(random_pixels, axis=0))
# this will contain our generated image
combination_image = K.variable(combination_image)

In [8]:
content_model = vgg16.VGG16(input_tensor=base_image, weights='imagenet', include_top=False)
style_model = vgg16.VGG16(input_tensor=style_reference_image, weights='imagenet', include_top=False)

generated_model = vgg16.VGG16(input_tensor=combination_image, weights='imagenet', include_top=False)


In [9]:

content_outputs = dict([(layer.name, layer.output) for layer in content_model.layers])
style_outputs = dict([(layer.name, layer.output) for layer in style_model.layers])

generated_outputs = dict([(layer.name, layer.output) for layer in generated_model.layers])

In [10]:

def content_loss(base, combination):
    return K.sum(K.square(combination - base))

loss = K.variable(0)

base_image_features = content_outputs['block5_conv2'][0]
combination_features = generated_outputs['block5_conv2'][0]
contentloss = content_loss(base_image_features, combination_features)

feature_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']


In [11]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [12]:
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4. * (pow(channels,2)) * (pow(size,2)))


styleloss = K.variable(0)

for layer_name in feature_layers:
    style_reference_features = style_outputs[layer_name][0]
    combination_features = generated_outputs[layer_name][0]
    styleloss = styleloss + style_loss(style_reference_features, combination_features)


In [13]:
alpha = 0.025; beta = 0.2
loss = alpha * contentloss + beta * styleloss

In [14]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
outputs += grads


In [15]:
f_outputs = K.function([combination_image], outputs)

def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values


In [16]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values


In [17]:
evaluator = Evaluator()


In [18]:
x = preprocess_image(base_image_path)

for i in range(0,iterations):
    print("epoch ",i)
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.grads, maxfun=20)
    img = deprocess_image(x.copy())
    fname = 'generated_%d.png' % i
    save_img(fname, img)

epoch  0
